ARTI308 - Machine Learning

# Lab 4: Data Quality Assessment & Preprocessing

In real-world machine learning projects, data is often:
- Incomplete (missing values)
- Noisy (outliers or random errors)
- Inconsistent (wrong formats, mixed units)

Before building any machine learning model, we must clean and prepare the data properly.

In this lab, we will apply practical preprocessing techniques step by step on the `vgsales` dataset.


In [ ]:
# Import Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

## 1. Load Dataset

In [ ]:
pd.set_option("display.max_columns", None)

df = pd.read_csv("vgsales.csv")
df.head(10)


## 2. Data Quality Assessment
### 2.1 Check Data Types
Data types must match the real meaning of each column.
For example:
- `Year` should be integer-like year values
- Sales columns should be numeric


In [ ]:
df.dtypes

We observe that `Year` is stored as float because there are missing values.
`Platform`, `Genre`, and `Publisher` can be stored as categorical features for better memory efficiency.
Correct data types support proper numerical analysis and preprocessing.


### 2.2 Convert Incorrect Data Types
We will convert:
- `Year` to nullable integer (`Int64`)
- `Platform`, `Genre`, and `Publisher` to categorical type


In [ ]:
df['Year'] = pd.to_numeric(df['Year'], errors='coerce').astype('Int64')

for col in ['Platform', 'Genre', 'Publisher']:
    df[col] = df[col].astype('category')

df.dtypes


Now `Year` is stored as nullable integer (`Int64`).
Categorical columns are properly encoded as `category`.
This improves type consistency and memory usage.


## 3. Handling Missing Values
### 3.1 Detect Missing Values
Missing values reduce data quality and can affect model performance.

In [ ]:
df.isna().sum()

The output shows whether any column contains missing values.
In `vgsales`, missing values are commonly found in `Year` and `Publisher`.
These missing values should be handled before modeling.


### 3.2 Demonstration: Introduce Artificial Missing Values
### Why?

Although the dataset already has missing values in some columns, we introduce artificial missing values in a numeric feature for imputation practice.

we will be running this line:

`df_missing.loc[0:5, 'Global_Sales'] = np.nan`


- `df_missing`: The pandas DataFrame you are modifying.

- `.loc[0:5, 'Global_Sales']`: This uses the label-based indexer to select specific rows and columns.

- `0:5`: Selects rows with index labels 0, 1, 2, 3, 4, and 5. In label-based indexing, the end index is inclusive.

- `'Global_Sales'`: Selects the column named 'Global_Sales'.

- `= np.nan`: Assigns the value np.nan (which stands for "Not a Number") to all the selected cells. This is the standard way to represent missing or null values in numerical columns in pandas. The column's data type remains numeric.


In [ ]:
df_missing = df.copy()
df_missing.loc[0:5, 'Global_Sales'] = np.nan
df_missing.isna().sum()


Now the `Global_Sales` column contains additional missing values.


In [ ]:
print("Original shape: ",df.shape)
print("After removing some values: ",df_missing.shape)

In [ ]:
df_missing.head(10)

### Strategy 1: Remove Records
This strategy removes records containing missing data.
It works well if the number of missing rows is small.

In [ ]:
df_removed = df_missing.dropna()
df_removed.shape

In [ ]:
df_removed.isna().sum()

The dataset now has fewer rows.
If only a small portion of data was missing, this method is acceptable.

However, removing too many rows can reduce model performance.

### Strategy 2: Mean Imputation

![Mean.png](Mean.png)

The mean represents the average value.
It is commonly used for normally distributed data.

In [ ]:
df_missing.head(10)


In [ ]:
df_imputed_mean = df_missing.copy()
df_imputed_mean['Global_Sales'] = df_imputed_mean['Global_Sales'].fillna(df_imputed_mean['Global_Sales'].mean())

df_imputed_mean.isna().sum()


In [ ]:
df_imputed_mean.head(10)


Missing values are now replaced with the average global sales.
This preserves dataset size but may reduce variability.
Mean imputation is sensitive to outliers.


### Strategy 3: Median Imputation

![median_formula_2.png](median_formula_2.png)
The median is more robust to outliers than the mean.
It is preferred for skewed data.

In [ ]:
df_imputed_median = df_missing.copy()
df_imputed_median['Global_Sales'] = df_imputed_median['Global_Sales'].fillna(df_imputed_median['Global_Sales'].median())

df_imputed_median.isna().sum()


In [ ]:
df_imputed_median.head(10)


Missing values are replaced with the middle value.
This approach is safer when data contains extreme values.

## 4. Handling Outliers
Outliers are extreme values that can distort models.
We will detect outliers in `Global_Sales` using the IQR method.


In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x=df['Global_Sales'])
plt.title("Boxplot of Global_Sales")
plt.show()


Points outside the whiskers represent potential outliers.
These extreme sales values may influence model predictions.


### Detect Outliers using IQR
**Method: Interquartile Range (IQR)**

The IQR method defines outliers as values outside:

`Q1 - 1.5×IQR`  and  `Q3 + 1.5×IQR`

In [ ]:
Q1 = df['Global_Sales'].quantile(0.25)
Q3 = df['Global_Sales'].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

outliers = df[(df['Global_Sales'] < lower) | (df['Global_Sales'] > upper)]
outliers.head(15)


The output displays records considered extreme based on statistical boundaries.
These may be valid high-value transactions or potential data errors.

### Remove Outliers
We remove values outside the acceptable range.

In [ ]:
df_no_outliers = df[(df['Global_Sales'] >= lower) & (df['Global_Sales'] <= upper)]
print("Original shape: ",df.shape)
print("After removing outliers: ",df_no_outliers.shape)


The dataset size is slightly reduced.
Removing outliers reduces distortion but may also remove important rare events.

#### Important Note on Removing Outliers

Not all outliers are errors.

Some extreme values may represent rare but important real-world events.  
For example, in a sales dataset, a very large transaction might correspond to a bulk corporate order or a seasonal promotion.  

If we remove such values blindly, we may lose valuable information and bias the analysis.

Before removing outliers, we should always ask:
- Is this value a data entry mistake?
- Or is it a valid but rare observation?

### Capping Outliers (Percentile Method)
Instead of removing outliers, we replace extreme values with percentile limits.

![percentile.png](percentile.png)

In [ ]:
lower_cap = df['Global_Sales'].quantile(0.05)
upper_cap = df['Global_Sales'].quantile(0.95)

df_capped = df.copy()
df_capped['Global_Sales'] = df_capped['Global_Sales'].clip(lower_cap, upper_cap)


## 5. Data Transformation ? Normalization
Normalization scales numerical features to a similar range.
This ensures that no feature influences the model simply because it has larger numerical values.

### Min-Max Normalization
Min-Max normalization rescales numerical values to a fixed range, usually between 0 and 1.

It works using the formula:

This method preserves the original distribution shape and relative ordering of values.

Min-Max normalization is especially useful for distance-based models such as:
- K-Nearest Neighbors (KNN)
- K-Means clustering
- Support Vector Machines (SVM)

These models rely on distance calculations, and if features are on very different scales, one feature can dominate the distance computation.


In [ ]:
df[['NA_Sales', 'EU_Sales']].head()


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_scaled = df[['NA_Sales', 'EU_Sales']].copy()

df_scaled[['NA_Sales', 'EU_Sales']] = scaler.fit_transform(df_scaled)

df_scaled.head()


After applying Min-Max normalization, all numerical values are scaled to the range between 0 and 1.

The smallest value in each feature becomes 0, and the largest becomes 1.
All other values are proportionally mapped between these two limits.

Importantly, normalization does NOT change the relative relationships between data points.
If one game originally had higher regional sales than another, it will still have a higher normalized value.


### Z-Score Normalization
Z-score standardization transforms the data so that:

- The mean of each feature becomes 0
- The standard deviation becomes 1

This is done by subtracting the mean and dividing by the standard deviation:

![zscore.png](zscore.png)

This method keeps the shape of the distribution but rescales it around zero.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_standardized = df[['NA_Sales', 'EU_Sales']].copy()

df_standardized[['NA_Sales', 'EU_Sales']] = scaler.fit_transform(df_standardized)

df_standardized.head()


After standardization, the numerical features are centered around 0.
Values above the original mean become **positive**, and values below the mean become **negative**.

The standard deviation of each feature becomes approximately 1, meaning the spread of the data is standardized.

This transformation is especially useful for:
- Linear regression
- Support Vector Machines (SVM)
- PCA

Because these models assume features are centered and scaled similarly.

## Check Correlation Before Applying PCA

we will check whether numerical features are correlated. If features are strongly correlated, they contain overlapping information.

- **Correlation close to 1**  → Strong positive linear relationship  
  (As one feature increases, the other also increases.)

- **Correlation close to -1** → Strong negative linear relationship  
  (As one feature increases, the other decreases.)

- **Correlation close to 0**  → Weak or no linear relationship  
  (The features do not move together in a predictable linear way.)

In such cases, dimensionality reduction using PCA is meaningful 
because we can combine correlated features into fewer components.

In [ ]:
plt.figure(figsize=(6,4))
sns.heatmap(df_standardized[['NA_Sales','EU_Sales']].corr(), 
            annot=True, cmap="coolwarm")
plt.title("Correlation Heatmap (Before PCA)")
plt.show()


The heatmap shows the correlation between the numerical features:

- The diagonal values are 1 because each feature is perfectly correlated with itself.
- The correlation between `NA_Sales` and `EU_Sales` is approximately *0.768*.

A positive value close to 1 indicates a **strong linear** relationship between the two features.

This means that `NA_Sales` and `EU_Sales` often increase together across many games.

Since PCA is most useful when features are correlated,
applying PCA here can meaningfully summarize the shared information.


## 6. Data Reduction – Principal Component Analysis (PCA)

Principal Component Analysis (PCA) is a dimensionality reduction technique.

Instead of working directly with the original features, PCA creates new features called **principal components**.

These components:

- Are linear combinations of the original features
- Are uncorrelated with each other
- Capture variance in descending order (from most important to least)

The first principal component (PC1) captures the largest possible variance in the dataset.

The second principal component (PC2) captures the next largest variance, while being orthogonal (perpendicular) to PC1.

This allows us to reduce dimensionality while retaining most of the important information in the data.

### Visual Intuition

Imagine we have two features:

X1 = NA_Sales  
X2 = EU_Sales  

If we plot the data points, they may look like this:

              X2
               |
               |
               |        *
               |      *
               |    *
               |  *
               | *
               |________________________ X1

Notice that the points follow a diagonal pattern.
This means the two features are correlated and contain overlapping information.

Instead of keeping both X1 and X2 separately,
PCA finds the direction where the data varies the most.

That direction becomes **Principal Component 1 (PC1)**.

              X2
               |
               |        *
               |      *
               |    *
               |  *
               | *
               |________________________ X1
                    \
                     \
                      \
                       \
                        -> PC1 (maximum variance direction)

PC2 is the direction perpendicular to PC1.

If most of the variation is along PC1,
then PC1 alone captures most of the dataset?s information.

In that case, we can reduce:

2 features -> 1 feature (PC1)

while keeping most of the variance.


In [ ]:
from sklearn.decomposition import PCA

X = df_standardized[['NA_Sales', 'EU_Sales']]

pca = PCA(n_components=2)
principal_components = pca.fit_transform(X)

print("Explained Variance Ratio:", pca.explained_variance_ratio_)


The `Explained Variance Ratio` indicates how much of the total information (variance) is captured by each principal component.

For example:
- If PC1 explains 85% of the variance, it means that one new feature already summarizes most of the dataset's information.
- If PC1 and PC2 together explain nearly 100%, then very little information is lost.

When most of the variance is captured by fewer components, dimensionality reduction is considered effective.

This helps simplify models, reduce computational cost, and sometimes improve generalization performance.

In [ ]:
# # In addition to the heatmap, we can visualize the relationship between two features using a scatter plot. 
# # A scatter plot allows us to observe whether the features exhibit a linear pattern, meaning whether they increase or decrease together in a consistent way.


# plt.figure(figsize=(6,4))
# plt.scatter(df_standardized['NA_Sales'], df_standardized['EU_Sales'])
# plt.xlabel("NA_Sales (Standardized)")
# plt.ylabel("EU_Sales (Standardized)")
# plt.title("Scatter Plot: NA_Sales vs EU_Sales")
# plt.show()


In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(principal_components[:,0], principal_components[:,1])
plt.title("PCA Projection")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.show()

Each point in this plot represents one game record.

The axes no longer represent the original features (NA_Sales and EU_Sales).
Instead:

- The horizontal axis represents Principal Component 1 (PC1).
- The vertical axis represents Principal Component 2 (PC2).

PC1 captures the direction of maximum variance in the data.
PC2 captures the second most important direction, perpendicular to PC1.

The spread of points along the horizontal direction indicates how much variation is captured by PC1.
If most of the spread appears horizontally, it suggests that PC1 captures most of the dataset's information.

This projection allows us to visualize high-dimensional data in a lower-dimensional space.


# Assignment

In this assignment, you will:
- **Task 1**
Identify data quality issues in the `vgsales` dataset.

- **Task 2**
Apply one missing value strategy and explain why.

- **Task 3**
Detect and handle outliers using IQR.

- **Task 4**
Normalize numerical features using both Min-Max and Z-score.

- **Task 5**
Apply PCA and interpret explained variance.


End of lab 4.
